# 코퍼스
- 네이버 영화 리뷰 데이터 다운로드(https://github.com/e9t/nsmc)
- pandas dataframe 으로 불러오기

In [3]:
import pandas as pd

data = pd.read_table('ratings_train.txt')[100:1100]
data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


# Konlpy 라이브러리 패키지를 활용한 토큰화
- step1: dataframe 토큰화 연습
- step2: 토큰화 함수 생성

In [2]:
from konlpy.tag import Okt, Kkma, Komoran, Hannanum

tokenizer = Okt()

In [9]:
# step 1
for idx, row in data.iterrows():
    doc = row['document']
    
    results = tokenizer.pos(doc)
    
    tokens = []
    for token, pos in results:
        # token, pos = r # ('더빙', 'Noun')
        if pos == 'Noun' or pos == 'Adjective':
            tokens.append(token)
    
    print('raw text:', doc)
    print('tokens:', tokens)
    
    break

raw text: 아 더빙.. 진짜 짜증나네요 목소리
tokens: ['더빙', '진짜', '짜증나네요', '목소리']


In [11]:
# step2: 함수 만들어 토큰화

def process_tokens(doc):
    results = tokenizer.pos(doc)

    tokens = []
    for r in results:
        token, pos = r
        if pos == 'Noun' or pos == 'Adjective':
            tokens.append(token)
    return tokens    

In [14]:
data['tokens'] = data['document'].apply(lambda x: process_tokens(x))
# data['tokens'] = data.apply(lambda row: process_tokens(row['document']), axis=1)
data = data.drop('document', axis=1) # drop의 기능 확인

data

,id,label,tokens
0,9976970,0,"[더빙, 진짜, 짜증나네요, 목소리]"
1,3819312,1,"[흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 가볍지]"
2,10265843,0,"[무재, 밓었, 다그, 래서, 추천]"
3,9045019,0,"[교도소, 이야기, 구먼, 솔직히, 재미, 없다, 평점, 조정]"
4,6483659,1,"[몬페, 의, 익살스런, 연기, 영화, 스파이더맨, 커스틴, 던스트, 이뻐]"
5,5403919,0,"[막, 걸음, 마, 세, 초등학교, 학년, 생인, 영화, 반개, 아까, 움]"
6,7797314,0,"[원작, 긴장감, 제대로]"
7,9443947,0,"[별, 반개, 아깝다, 욕, 이응경, 길용우, 생활, 년, 정말, 발, 해도, 그것..."
8,7156791,1,"[액션, 없는데도, 재미, 있는, 몇, 안되는, 영화]"
9,5912145,1,"[왜케, 평점, 낮은건데, 꽤, 볼, 데, 헐리우드, 화려함에만, 있나]"



# 임베딩
- gensim 패키지 설치(pip install gensim)
- Word2Vec, FastText 객체 생성

In [15]:
# library : gensim
from gensim.models import Word2Vec, FastText

tokenized_data = data['tokens']

# train word2vec model
wvmodel = Word2Vec(
    sentences = tokenized_data,
    vector_size = 2,
    min_count = 1,
    window = 5,
    sg = 1
)

# FastText 임베딩
ftmodel = FastText(
    sentences=tokenized_data,
    vector_size=2,
    min_count=1,
    window=5,
    sg=1
)

# 유사도 검색

In [16]:
# 가장 유사한 벡터 찾기
for tokens in tokenized_data:
    print(tokens)
    
    for token in tokens:
        print('word2vec:', token, wvmodel.wv[token], wvmodel.wv.most_similar(token, topn=3))
        print('fasttext:', token, ftmodel.wv[token], ftmodel.wv.most_similar(token, topn=3))
        print('\n')
        
    break

['더빙', '진짜', '짜증나네요', '목소리']
word2vec: 더빙 [-0.00974282  0.17400204] [('가볍지', 0.9946810007095337), ('이야기', 0.994314968585968), ('원작', 0.9808803200721741)]
fasttext: 더빙 [-0.05973839 -0.03152505] [('던스트', 0.9999072551727295), ('화려함에만', 0.9995203018188477), ('있는', 0.9978805184364319)]


word2vec: 진짜 [0.24927534 0.46211368] [('연기', 0.9997234344482422), ('감금', 0.9989298582077026), ('밓었', 0.996440052986145)]
fasttext: 진짜 [0.18167806 0.34088123] [('납치', 0.9999974370002747), ('액션', 0.9994968771934509), ('래서', 0.998386800289154)]


word2vec: 짜증나네요 [-0.4062044   0.22522794] [('구먼', 0.9996954798698425), ('영화', 0.9992628693580627), ('안되는', 0.9945798516273499)]
fasttext: 짜증나네요 [-0.07428907 -0.11761305] [('정말', 0.9993209838867188), ('안되는', 0.9935410618782043), ('스파이더맨', 0.9721710681915283)]


word2vec: 목소리 [-0.20603311  0.04192438] [('줄', 0.9999739527702332), ('드라마', 0.9911259412765503), ('안되는', 0.9797953963279724)]
fasttext: 목소리 [-0.00038061  0.15203878] [('보고', 0.99619460105896), ('가볍지', 0.98986840

# Word2Vec vs. FastText
- OOV(Out of Vocabulary) 현상

In [23]:
# word2vec
 
sample_token = ['목소리']
try:
    print('token:', sample_token, 'Word2Vec:', wvmodel.wv.most_similar(sample_token, topn=3))
except KeyError as e:
    print('Word2Vec: error handling ', e)
    

token: ['목소리'] Word2Vec: [('줄', 0.9999739527702332), ('드라마', 0.9911259412765503), ('안되는', 0.9797953963279724)]


In [24]:
# FastText

try:
    print('token:', sample_token, 'FastText:', ftmodel.wv.most_similar(sample_token, topn=3))
except KeyError as e:
    print('FastText: error handling ', e)

token: ['목소리'] FastText: [('보고', 0.99619460105896), ('가볍지', 0.9898684024810791), ('무재', 0.9897528290748596)]
